In [1]:
import xml.etree.ElementTree as ET
import sqlite3
import pandas as pd

# See https://github.com/sillsdev/lift-standard/blob/master/lift_13.pdf

In [2]:
DATABASE_NAME = ":memory:"
con = sqlite3.connect(DATABASE_NAME)
cur = con.cursor()
cur.executescript("""
    PRAGMA foreign_keys = ON;
    
    CREATE TABLE IF NOT EXISTS lexemes(
        rowid INTEGER PRIMARY KEY,
        lemma TEXT,
        morpheme_type TEXT
    );
    
    CREATE TABLE IF NOT EXISTS spellings(
        form TEXT,
        lexeme INTEGER,
        FOREIGN KEY(lexeme) REFERENCES lexemes(rowid)
    );

    CREATE TABLE IF NOT EXISTS senses(
        rowid INTEGER PRIMARY KEY,
        gloss TEXT,
        lexeme INTEGER,
        part_of_speech TEXT,
        FOREIGN KEY(lexeme) REFERENCES lexemes(rowid)
    );

    CREATE TABLE IF NOT EXISTS sense_grammatical_info(
        name TEXT,
        value TEXT,
        sense INTEGER,
        FOREIGN KEY(sense) REFERENCES senses(rowid)
    );
""")

In [3]:
LEXICON_XML = "xml/xml.lift"
tree = ET.parse(LEXICON_XML)
root = tree.getroot()
root

<Element 'lift' at 0x0000026CA78738D0>

In [4]:
%%time

for entry in root.findall('entry'):
    lexeme_form = entry.find('./lexical-unit/form')
    lemma = lexeme_form.find('text').text
    morpheme_type = entry.find('./trait[@name="morph-type"]').attrib['value']
    
    cur.execute(f"INSERT INTO lexemes VALUES (NULL, '{lemma}', '{morpheme_type}')")
    con.commit()
    
    lexeme_id = cur.lastrowid
    cur.execute(f"INSERT INTO spellings VALUES ('{lemma}', {lexeme_id})")
    for variant_form in entry.findall('./variant/form/text'):
        cur.execute(f"INSERT INTO spellings VALUES ('{variant_form.text}', {lexeme_id})")
    con.commit()

    for sense in entry.findall('sense'):
        gloss = sense.find('./gloss/text').text
        part_of_speech = sense.find('grammatical-info').attrib['value']
        cur.execute(f"INSERT INTO senses VALUES (NULL, '{gloss}', {lexeme_id}, '{part_of_speech}')")
        con.commit()
        sense_id = cur.lastrowid
        for trait in sense.findall('./grammatical-info/trait'):
            name = trait.attrib['name']
            value = trait.attrib['value']
            cur.execute(f"INSERT INTO sense_grammatical_info VALUES ('{name}', '{value}', {lexeme_id})")
        con.commit()

lexemes_df = pd.read_sql_query("SELECT * FROM lexemes", con)
spellings_df = pd.read_sql_query("SELECT * FROM spellings", con)
senses_df = pd.read_sql_query("SELECT * FROM senses", con)
gramm_df = pd.read_sql_query("SELECT * FROM sense_grammatical_info", con)
con.close()

display(lexemes_df)
display(spellings_df)
display(senses_df)
display(gramm_df)

,rowid,lemma,morpheme_type
0,1,r-,prefix
1,2,sayù-,prefix
2,3,*eʔ,bound stem
3,4,e-,prefix
4,5,*ǫ:méh,bound stem
5,6,*werǫ́hs,bound stem


,form,lexeme
0,r-,1
1,sayù-,2
2,*eʔ,3
3,e-,4
4,í:-,4
5,*ǫ:méh,5
6,*werǫ́hs,6


,rowid,gloss,lexeme,part_of_speech
0,1,3.m.sg,1,Verb
1,2,3.m.sg.3.n.sg,2,Verb
2,3,go,3,Verb
3,4,PROTHETIC,4,Verb
4,5,person,5,Verb
5,6,trick,6,Verb


,name,value,sense
0,type,inflAffix,1
1,inflection-feature,{sbj}[sbj:[gen:m pers:3 num:sg]],1
2,type,inflAffix,2
3,inflection-feature,{obj}[sbj:[num:sg pers:3 gen:m] obj:[gen:n per...,2
4,type,inflAffix,4


CPU times: total: 31.2 ms
Wall time: 40.4 ms
